# More Pandas

![panda](http://res.freestockphotos.biz/thumbs/3/3173-illustration-of-a-giant-panda-eating-bamboo-th.png)

Suppose you were interested in opening an animal shelter. To inform your planning, it would be useful to analyze data from other shelters to understand their operations. In this lecture, we'll analyze animal outcome data from the Austin Animal Center.  

## Objectives

- Apply and use `.map()`, `apply()`, and `.applymap()` from the `pandas` library
- Apply and use `.where()` and `.select()` from the `numpy` library
- Use lambda functions in coordination with the above functions
- Explain what a groupby object is and split a DataFrame using `.groupby()`

In [ ]:
import numpy as np
import pandas as pd
import requests
from matplotlib import pyplot as plt

%matplotlib inline

# These next lines ensure that the notebook
# stays current with respect to active .py files.
# See here:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html

%load_ext autoreload
%autoreload 2

## Loading the Data

Let's take a moment to download and to examine the [Austin Animal Center data set](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Outcomes/9t4d-g238/data). 

We can also ingest the data right off the web, as we do below. The code below will load data for the last 1000 animals to leave the center. 

In [ ]:
url = 'https://data.austintexas.gov/resource/9t4d-g238.json'
response = requests.get(url)
animals = pd.DataFrame(response.json())

# Exploratory Data Analysis (EDA)

Exploring a new dataset is essential for understanding what it contains. This will generate ideas for processing the data and questions to try to answer in futher analysis.

## Inspecting the Data

Let's take a look at a few rows of data.

In [ ]:
animals.head()

The `info()` and `describe()` provide a useful overview of the data.

In [ ]:
animals.info()

In [ ]:
animals.describe()

In [ ]:
# Use value counts to check a categorical feature's distribution

animals['color'].value_counts()

Now that we have a sense of the data available to us, we can focus in on some more specific questions to dig into. These questions may or may not be directly relevant to your goal (e.g. helping plan a new shelter), but will always help you gain a better understanding of your data.

In your EDA notebooks, **markdown** will be especially helpful in tracking these questions and your methods of answering the questions.  

## Question 1: What animal types are in the dataset?

We can then begin thinking about what parts of the DataFrame we need to answer the question.

    What features do we need?
     - "animal_type"
    What type of logic and calculation do we perform?
     - Let's use `.value_counts()` to count the different animal types
    What type of visualization would help us answer the question?
     - A bar chart would be good for this purpose

In [ ]:
animals['animal_type'].value_counts()

In [ ]:
fig, ax = plt.subplots()

ax.barh(animals['animal_type'].value_counts().index,
        width=animals['animal_type'].value_counts().values)
ax.set_xlabel('count');

In [ ]:
animals['animal_type'].hist()

Questions lead to other questions. For the above example, the visualization raises the question...

## Question 2: What "Other" animals are in the dataset?

To find out, we need to know whether the type of animal for "Other" is in our dataset - and if so, where to find it.   
    
**Discussion**: Where might we look to find animal types within the Other category?

<details>
    <summary>
        Answer
    </summary>
        The breed column.
</details>

In [ ]:
# Your exploration here

Let's use that column to answer our question.

In [ ]:
animals[animals['animal_type'] == 'Other']['breed'].value_counts()

## Question 3: How old are the animals in our dataset?

Let's try to answer this with the `age_upon_outcome` variable to learn some new `pandas` tools.

In [ ]:
animals['age_upon_outcome'].value_counts()

### `Series.map()` and `Series.apply()`

The `.map()` method applies a transformation to every entry in the Series. This transformation  "maps" each value from the Series to a new value. A transformation can be defined by a function, Series, or dictionary - usually we'll use functions.

The `.apply()` method is similar to the `.map()` method for Series, but can only use functions. It has more powerful uses when working with DataFrames.

In [ ]:
def one_year(age):
    if age == '1 year':
        return '1 years'
    else:
        return age

In [ ]:
animals['new_age1'] = animals['age_upon_outcome'].map(one_year)
animals['new_age1'].value_counts()

### Slower Than `numpy`

In general, `np.where()` and `np.select()` are faster:

In [ ]:
animals['new_age2'] = np.where(animals['age_upon_outcome'] == '1 year',
                              '1 years', animals['age_upon_outcome'])
animals['new_age2']

In [ ]:
(animals['new_age1'] != animals['new_age2']).sum()

In [ ]:
%timeit animals['new_age1'] = animals['age_upon_outcome'].map(one_year)

In [ ]:
%timeit animals['new_age2'] = np.where(animals['age_upon_outcome'] == '1 year', \
                              '1 years', animals['age_upon_outcome'])

### More Sophisticated Mapping

Let's use `.map()` to turn sex_upon_outcome into a category with three values (called **ternary**): male, female, or unknown. 

First, explore the unique values:

In [ ]:
animals['sex_upon_outcome'].unique()

In [ ]:
def sex_mapper(status):
    if status in ['Neutered Male', 'Intact Male']:
        return 'Male'
    elif status in ['Spayed Female', 'Intact Female']:
        return 'Female'
    else:
        return 'Unknown'

In [ ]:
animals['new_sex1'] = animals['sex_upon_outcome'].map(sex_mapper)
animals['new_sex1']

Again, `numpy` will be faster:

In [ ]:
conditions = [animals['sex_upon_outcome'] == 'Neutered Male',
             animals['sex_upon_outcome'] == 'Intact Male',
             animals['sex_upon_outcome'] == 'Spayed Female',
             animals['sex_upon_outcome'] == 'Intact Female',
             animals['sex_upon_outcome'] == 'Unknown',
             animals['sex_upon_outcome'] == 'NULL']

choices = ['Male', 'Male', 'Female', 'Female', 'Unknown', 'Unknown']

In [ ]:
animals['new_sex2'] = np.select(conditions, choices)
animals['new_sex2']

In [ ]:
(animals['new_sex1'] != animals['new_sex2']).sum()

In [ ]:
%timeit animals['new_sex1'] = animals['sex_upon_outcome'].map(sex_mapper)

In [ ]:
%timeit animals['new_sex2'] = np.select(conditions, choices)

### Lambda Functions

Simple functions can be defined just when you need them, when you would call the function. These are called **lambda functions**. These functions are **anonymous** and disappear immediately after use.

Let's use a lambda function to get rid of 'Other' in the "animal_type' column.

In [ ]:
animals[animals['animal_type'] == 'Other']

In [ ]:
animals['animal_type'].value_counts()

In [ ]:
animals['animal_type'].map(lambda x: np.nan if x == 'Other' else x).value_counts()

## Methods for Re-Organizing DataFrames: `.groupby()`

The `.groupby()` method lets you group together subsets of data so you can apply functions to groups.

It is most useful when we have numeric data that we want to **aggregate**, meaning that we want to combine multiple observations together into a new, summary observation. 

Let's create a numeric variable by turning "age_upon_outcome" into a number of days.

This method is similar to the GROUP BY statement in SQL, which we'll discuss soon.

In [ ]:
animals['age_split'] = animals['new_age1'].str.split(" ")
animals['age_split']

In [ ]:
def count_days(x):
    """
    This function will convert ages into numbers of days.
    """
    if x[-1] == 'days':
        return int(x[0])
    elif x[-1] == 'weeks':
        return int(x[0]) * 7
    elif x[-1] == 'months':
        return int(x[0]) * 30
    elif x[-1] == 'years':
        return int(x[0]) * 365
    else:
        return np.nan

In [ ]:
animals['age_days'] = animals['age_split'].map(count_days).astype(float)
animals['age_days']

In [ ]:
animals.groupby('animal_type').mean()

Notice the object type [DataFrameGroupBy](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) object. 

In [ ]:
type(animals.groupby('animal_type'))

### .groups and .get_group()

In [ ]:
# This retuns a dictionary with each group 
# indexed by the group name, along with the 
# row indices for each animal in the group.

animals.groupby('animal_type').groups

In [ ]:
animals.groupby('animal_type').get_group('Dog')

We can group by multiple columns, and also return a DataFrameGroupBy object

In [ ]:
animals.groupby(['animal_type', 'outcome_type'])

In [ ]:
animals.groupby(['animal_type', 'outcome_type']).groups.keys()

In [ ]:
# We can then get a specific group, such as cats that were adopted

animals.groupby(['animal_type', 'outcome_type']).get_group(('Cat', 'Adoption'))

## Aggregating

In [ ]:
# Just like with single axis groups, we can aggregate on multiple axes

animals.groupby(['animal_type', 'outcome_type']).mean()

## `.agg()`

Use the `.agg()` method to specify in more detail the variables you want to aggregate and the aggregation functions you want to use.

In [ ]:
animals.groupby(['animal_type', 'outcome_type']).agg({'age_days': ['mean', 'max']})

## Level Up: `.applymap()`

`.applymap()` is used to apply a transformation to each element of a DataFrame.

In [ ]:
# This line will apply the base `type()` function to 
# all entries of the DataFrame.

animals.applymap(type)